# Workflow: processing, searching and retrieving observations

This tutorial demonstrates how OpenGHG can be used to process new measurement data, search the data present and to retrieve this for analysis and visualisation.

### Check installation

This tutorial assumes that you have installed `openghg`. To ensure install has been successful you can open an `ipython` console and try to import this module.

In a terminal type:

```bash
$ ipython
```

Then import `openghg` and print the version string associated with the version you have installed. If you get something like the below `openghg` is installed correctly.

```ipython
In [1]: import openghg
In [2]: openghg.__version__
Out[2]: '0.0.1'
```

If you get an ``ImportError`` please go back to the [install section of the documentation](https://docs.openghg.org/install.html).

### Jupyter notebooks

If you haven't used Jupyter notebooks before please see [this introduction](https://realpython.com/jupyter-notebook-introduction/).

## 1. Setting up an object store

The OpenGHG platform uses what's called an *object store* to save data. Any saved data has been processed into a standardised format, assigned universally unique identifiers (UUIDs) and stored alongside associated metadata (such as site and species details). Storing data in this way allows for fast retrieval and efficient searching.

When using OpenGHG on a local machine the location of the object store is set using an `OPENGHG_PATH` environment variable (explained below) and this can be any directory on your local system.

For this tutorial, we will create a temporary object store which we can add data to. This path is fine for this purpose but as it is a temporary directory it may not survive a reboot of the computer. 

The `OPENGHG_PATH` environment variable can be set up in the following way.

In [1]:
import os
import tempfile

tmp_dir = tempfile.TemporaryDirectory()
os.environ["OPENGHG_PATH"] = tmp_dir.name   # temporary directory

When creating your own longer term object store we recommend a path such as ``~/openghg_store`` which will create the object store in your home directory in a directory called ``openghg_store``. If you want this to be a permanent location this can be added to your "~/.bashrc" or "~/.bash_profile" file depending on the system being used. e.g. as

```bash
 export OPENGHG_PATH="$HOME/openghg_store"
```

## 2. Adding and standardising data

### Data types

Within OpenGHG there are several data types which can be processed and stored within the object store. This includes data from the AGAGE, DECC, NOAA, LondonGHG, BEAC2ON networks.

When uploading a new data file, the data type must be specified alongside some additional details so OpenGHG can recognise the format and the correct standardisation can occur. The details needed will vary by the type of data being uploaded but will often include the measurement reference (e.g. a site code) and the name of any network.

For the full list of accepted observation inputs and data types, there is a summary function which can be called:

In [2]:
from openghg.standardise import summary_data_types

summary = summary_data_types()

## UNCOMMENT THIS CODE TO SHOW ALL ENTRIES
# import pandas as pd; pd.set_option('display.max_rows', None)

summary

/user/home/rt17603/work/environments/openghg_env/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


,Site code,Long name,Data type,Platform
0,BTT,BT Tower,BTT,surface site
1,BSD,bilsdale,CRDS,surface site
2,HFD,heathfield,CRDS,surface site
3,KVH,Killearn Village Hall,CRDS,surface site
4,RGL,ridgehill,CRDS,surface site
...,...,...,...,...
157,WILSHIRECRESTELEMENTARYSCHOOL,Wilshire Crest Elementary School,BEACO2N,surface site
158,NPL,National Physical Laboratory,NPL,surface site
159,,,AQMESH,surface site
160,,,GLASGOW_PICARRO,surface site


Note: there may be multiple data types applicable for a give site. This is can be dependent on various factors including the instrument type used to measure the data e.g. for Bilsdale ("BSD"):

In [3]:
summary[summary["Site code"] == "BSD"]

,Site code,Long name,Data type,Platform
1,BSD,bilsdale,CRDS,surface site
10,BSD,bilsdale,GCWERKS,surface site


### DECC network

We will start by adding data to the object store from a surface site within the DECC network. Here we have accessed a subset of data from the Bilsdale site (site code "BSD") in the UK.

In [4]:
bsd_filepaths = ["../data/DECC/bsd.picarro.1minute.42m.min.dat", 
                 "../data/DECC/bsd.picarro.1minute.108m.min.dat", 
                 "../data/DECC/bsd.picarro.1minute.248m.min.dat"]

As this data is measured in-situ, this is classed as a surface site and we need to use the `ObsSurface` class to interpret this data. We can pass our list of files to the `read_file` method associated within the `ObsSurface` class, also providing details on:
 - site code - `"BSD"` for Billsdale
 - type of data we want to process, known as the data type - `"CRDS"`
 - network - `"DECC"`

This is shown below:

In [5]:
from openghg.store import ObsSurface

decc_results = ObsSurface.read_file(filepath=bsd_filepaths, data_type="CRDS", site="bsd", network="DECC")

Processing: bsd.picarro.1minute.248m.min.dat: 100%|██████████| 3/3 [00:01<00:00,  2.33it/s]


In [6]:
print(decc_results)

defaultdict(<class 'dict'>, {'processed': {'bsd.picarro.1minute.42m.min.dat': {'ch4': 'd2595d46-81b9-453e-9eb6-c2247255ac64', 'co2': '6118bd7c-175c-4604-b400-536264e2167d', 'co': 'f665fc44-ebea-498a-bf4e-c21d7feaa64a'}, 'bsd.picarro.1minute.108m.min.dat': {'ch4': 'c4aeaaaf-b674-49ec-ba3a-4fd0e58c0433', 'co2': 'f88335ad-65ba-4d60-8520-11f26adec1c0', 'co': '83b52132-782c-4724-9787-25297ec2e09e'}, 'bsd.picarro.1minute.248m.min.dat': {'ch4': '65d92c78-61f3-4624-8c74-9efa0ed63314', 'co2': 'da9b6820-7b97-4494-b38a-223c2d12f2c6', 'co': '6675c41d-05fc-42f9-98ca-59fe4408ff42'}}})


Here this extracts the data (and metadata) from the supplied files, standardises them and adds these to our created object store.

The returned `decc_results` will give us a dictionary of how the data has been stored. The data itself may have been split into different entries, each one stored with a unique ID (UUID). Each entry is known as a *Datasource* (see below for a note on Datasources). The `decc_results` output includes details of the processed data and tells us that the data has been stored correctly. This will also tell us if any errors have been encountered when trying to access and standardise this data.

### AGAGE data

Another data type which can be added is data from the AGAGE network. The functions that process the AGAGE data expect data to have an accompanying precisions file. For each data file we create a tuple with the data filename and the precisions filename. *Note: A simpler method of uploading these file types is planned.*

We must create a `tuple` associated with each data file to link this to a precision file:

```python
list_of_tuples = [(data1_filepath, precision1_filepath), (data2_filepath, precision2_filepath), ...]
```

The data being uploaded here is from the Cape Grim station in Australia, site code "CGO".

In [7]:
agage_tuples = [('../data/AGAGE/capegrim-medusa.18.C', '../data/AGAGE/capegrim-medusa.18.precisions.C')]

We can add these files to the object store in the same way as the DECC data by including the right keywords:
 - site code - `"CGO"` for Cape Grim
 - data type - `"GCWERKS"`
 - network - `"AGAGE"`

In [8]:
from openghg.store import ObsSurface

agage_results = ObsSurface.read_file(filepath=agage_tuples, data_type="GCWERKS", site="CGO", network="AGAGE")

Processing: capegrim-medusa.18.C: 100%|██████████| 1/1 [00:02<00:00,  2.32s/it]


When viewing `agage_results` there will be a large number of Datasource UUIDs shown due to the large number of gases in each data file

In [9]:
agage_results

defaultdict(dict,
            {'processed': {'capegrim-medusa.18.C': {'nf3_70m': '435b79df-8fc7-4f44-9e18-2e7f0b8599cf',
               'cf4_70m': 'b1d08105-e7b0-4415-8ec4-1c7c7ac17efc',
               'c2f6_70m': '08c9b551-b8a9-4fa9-a85a-2661092ef378',
               'c3f8_70m': 'dd95905a-47e8-4cd4-84fa-4d498b28b291',
               'c4f8_70m': '9885df74-af82-407a-b309-045ed114d611',
               'c4f10_70m': '7f35663a-e861-4cb5-baf1-8876bd97ba5a',
               'c6f14_70m': '30446f64-af0b-41a2-a059-e2ed35065bc2',
               'sf6_70m': '32231c9c-42ad-44e7-b3cd-b25b3180a297',
               'so2f2_70m': 'c3aa230d-0d46-4c64-b2de-763a217757db',
               'sf5cf3_70m': 'c75004fc-d077-4cdc-9697-b5da029aa97f',
               'hfc23_70m': '7b46a842-b31a-4ea1-8959-7a70d7d9a104',
               'hfc32_70m': '2ecd7ff4-ad9f-4802-b7a0-afa1ac47bfab',
               'hfc125_70m': '1fcd8921-45b0-48d0-a3b8-89e688faa3b6',
               'hfc134a_70m': '61d0d67b-fe74-4510-8f33-63022e57c70c'

#### A note on Datasources

Datasources are objects that are stored in the object store (++add link to object store notes++) that hold the data and metadata associated with each measurement we upload to the platform.

For example, if we upload a file that contains readings for three gas species from a single site at a specific inlet height OpenGHG    will assign this data to three different Datasources, one for each species. Metadata such as the site, inlet height, species, network etc are stored alongside the measurements for easy searching. 

Datasources can also handle multiple versions of data from a single site, so if scales or other factors change multiple versions may be stored for easy future comparison.

## 3. Searching for data

### Visualising the object store

Now that we have added data to our created object store, we can view the objects within it in a simple force graph model. To do this we use the `view_store` function from the `objectstore` submodule. Note that the cell may take a few moments to load as the force graph is created.

In the force graph the central blue node is the `ObsSurface` node. Associated with this node are all the data processed by it. The next node in the topology are networks, shown in green. In the graph you will see `DECC` and `AGAGE` nodes from the data files we have added. From these you'll see site nodes in red and then individual datasources in orange.

*Note: The object store visualisation created by this function is commented out here and won't be visible in the documentation but can be uncommented and run when you use the notebook version.*


In [10]:
from openghg.objectstore import visualise_store

# visualise_store()

Now we know we have this data in the object store we can search it and retrieve data from it.

### Searching the object store

We can search the object store by property using the `search(...)` function.

For example we can find all sites which have measurements for carbon tetrafluoride ("cf4") using the `species` keyword:

In [11]:
from openghg.retrieve import search
search(species="cf4")

Site: CGO
---------
cf4 at 70m


We could also look for details of all the data measured at the Billsdale ("BSD") site using the `site` keyword:

In [12]:
search(site="bsd")

Site: BSD
---------
ch4 at 42m, 108m, 248m
co2 at 42m, 108m, 248m
co at 42m, 108m, 248m


For this site you can see this contains details of each of the species as well as the inlet heights these were measured at.

## 4. Retrieving data 

To retrieve the standardised data from the object store there are several functions we can use which depend on the type of data we want to access.

To access the surface data we have added so far we can use the `get_obs_surface` function and pass keywords for the site code, species and inlet height to retrieve our data.

In this case we want to extract the carbon monoxide ("co") data from the Bilsdale data ("BSD") site measured at the "248m" inlet:

In [13]:
from openghg.retrieve import get_obs_surface

obs_data = get_obs_surface(site="bsd", species="co", inlet="248m")

If we view our returned `obs_data` variable this will contain:

 - `data` - The standardised data (accessed using e.g. `obs_data.data`). This is returned as an [xarray Dataset](https://xarray.pydata.org/en/stable/generated/xarray.Dataset.html).
 - `metadata` - The associated metadata (accessed using e.g. `obs_data.metadata`).

In [14]:
obs_data

ObsData(data=<xarray.Dataset>
Dimensions:                    (time: 142)
Coordinates:
  * time                       (time) datetime64[ns] 2014-01-30T11:12:30 ... ...
Data variables:
    mf                         (time) float64 202.4 203.2 205.1 ... 114.5 114.2
    mf_variability             (time) float64 5.265 6.307 8.518 ... 7.339 5.405
    mf_number_of_observations  (time) float64 26.0 26.0 25.0 ... 23.0 24.0 23.0
Attributes: (12/23)
    data_owner:           Simon O'Doherty
    data_owner_email:     s.odoherty@bristol.ac.uk
    inlet_height_magl:    248m
    comment:              Cavity ring-down measurements. Output from GCWerks
    long_name:            bilsdale
    conditions_of_use:    Ensure that you contact the data owner at the outse...
    ...                   ...
    sampling_period:      60
    inlet:                248m
    port:                 9
    type:                 air
    network:              decc
    scale:                WMO-X2014A, metadata={'site': 'bsd'

First we tell `matplotlib` that we are plotting inside a Jupyter notebook, this ensures a plot with controls is created.

In [15]:
%matplotlib notebook

In [16]:
example_data = obs_data.data
mol_frac = example_data.mf
mol_frac.plot()

<IPython.core.display.Javascript object>

---

#### Cleanup

If you used the `tmp_dir` as a location for your object store at the start of the tutorial you can run the cell below to remove any files that were created to make sure any persistant data is refreshed when the notebook is re-run.

In [17]:
tmp_dir.cleanup()